List of thing to do:

- Simple square example
- Pointers
- Basic C functions (Memcpy, Malloc)
- Kernels
- Launching a kernel
- Maybe: ways to split our information

# Elevando al cuadrado con CUDA

Nuestra primer misión en CUDA es expicar ciertas sutilezas, para esto iniciaremos con un pequeño ejemplo.

Supongamos que queremos elevar al cuadrado los números del 1 al 100. Un programa en python podríamos hacerlo de la siguiente manera.

- Generamos un arreglo con los números del 1 al 100.
- Utilizando un ciclo **for** elevamos cada uno de estos números y los metemos en un nuevo arreglo.

Por último lo imprimos para mostrarlo al lector.

In [1]:
enteros = range(1,101)
cuadrados = []

for i in enteros:
    cuadrados.append(i*i)

print(cuadrados)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801, 10000]


Como podemos ver, también hemos calculado el tiempo que se toma en hacer todo el proceso. Analicemos un poco nuestro algoritmo; el programa hace lo siguiente

- Lee un número del arreglo.
- Lo multiplica por sí mismo.
- Guarda el resultado en un nuevo arreglo.
- Si hay un siguiente número se regresa al punto inicial, si no termina.

Supongamos a manera de esquema que cada multiplicación en nuestro procesador toma **2 ns** en realizarse, como estamos haciendo 100 multiplicaciones, entonces nos tomaría **200 ns** obtener todos y cada uno de los cuadrados que queremos.

¿Cómo podríamos hacer esto en paralelo?

Bueno, tal vez utilizando 100 trabajadores (a los cuales de aquí en adelante llamaremos threads o hilos) y haciendo que cada uno se encargue de leer un número del arreglo de entrada, multiplicarlo por sí mismo y escribirlo dentro de otro arreglo. Si vemos lo que hace un sólo thread no podríamos notar cuál es la diferencia entre el algoritmo en Python (algoritmo escrito en serie) y el algoritmo en parelelo. La principal diferencia radica en que en el algoritmo serial un solo thread se encarga de multiplicar todos los números, avanzando de uno en uno, mientras que en el algoritmo en paralelo le damos un número a cada uno de los 100 trabajadores y los hacemos multiplicarlos por sí mismos. Si cada uno de estos trabajadores tardara **10 ns** en hacer una multiplicación ¿Cuánto se tardarían en multiplicar los 100 números? Exacto, **10 ns**.

Ahora veamos cómo quedaría éste código escrito en CUDA.

```
#include <stdio.h>

__global__ void cube(float * d_out, float * d_in){
	int idx = threadIdx.x;
    float f = d_in[idx];
    d_out[idx] = f*f*f;
}

int main(int argc, char ** argv) {
	const int ARRAY_SIZE = 100;
	const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

	// generate the input array on the host
	float h_in[ARRAY_SIZE];
	for (int i = 0; i < ARRAY_SIZE; i++) {
		h_in[i] = float(i);
	}
	float h_out[ARRAY_SIZE];

	// declare GPU memory pointers
	float * d_in;
	float * d_out;

	// allocate GPU memory
	cudaMalloc((void**) &d_in, ARRAY_BYTES);
	cudaMalloc((void**) &d_out, ARRAY_BYTES);

	// transfer the array to the GPU
	cudaMemcpy(d_in, h_in, ARRAY_BYTES, cudaMemcpyHostToDevice);

	// launch the kernel
	cube<<<1, ARRAY_SIZE>>>(d_out, d_in);

	// copy back the result array to the CPU
	cudaMemcpy(h_out, d_out, ARRAY_BYTES, cudaMemcpyDeviceToHost);

	// print out the resulting array
	for (int i =0; i < ARRAY_SIZE; i++) {
		printf("%f", h_out[i]);
		printf(((i % 4) != 3) ? "\t" : "\n");
	}

	cudaFree(d_in);
	cudaFree(d_out);

	return 0;
}```